# 中等项目1：完整客户流失预测系统

## 项目描述

构建一个完整的客户流失预测系统，包括数据探索、特征工程、模型训练、评估和结果分析。这是一个完整的数据科学项目流程。

## 学习目标

通过本项目，你将学会：
1. 如何进行完整的数据探索（EDA）
2. 如何进行特征工程和预处理
3. 如何训练和优化模型
4. 如何全面评估模型性能
5. 如何分析特征重要性

## 项目流程

1. 数据探索（EDA）：统计分析、可视化、相关性分析
2. 数据预处理：缺失值处理、异常值处理、特征编码
3. 特征工程：特征选择、特征创建、特征变换
4. 模型训练：训练多个模型，使用交叉验证
5. 模型评估：多种评估指标、ROC曲线、混淆矩阵
6. 结果分析：特征重要性分析、业务建议


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            roc_auc_score, roc_curve, precision_recall_curve, auc)
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
%matplotlib inline

print("环境准备完成！")


## 1. 数据生成与加载

生成模拟客户流失数据。在实际项目中，你需要从数据库或文件中加载真实数据。


In [ ]:
# 生成模拟客户流失数据
# 特征包括：年龄、收入、使用时长、投诉次数、服务次数、满意度等
np.random.seed(42)
n_samples = 2000

# 生成特征
data = {
    'age': np.random.normal(35, 10, n_samples).clip(18, 80),
    'income': np.random.normal(50000, 15000, n_samples).clip(20000, 150000),
    'usage_months': np.random.normal(24, 12, n_samples).clip(1, 60),
    'complaints': np.random.poisson(2, n_samples),
    'service_calls': np.random.poisson(3, n_samples),
    'satisfaction': np.random.uniform(1, 10, n_samples),
    'contract_type': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_samples),
    'payment_method': np.random.choice(['Electronic', 'Mailed check', 'Bank transfer'], n_samples)
}

df = pd.DataFrame(data)

# 生成目标变量（流失）：基于特征的逻辑组合
# 流失概率与投诉次数、满意度、合同类型等相关
churn_prob = (
    0.1 * (df['complaints'] / 10) +
    0.2 * (1 - df['satisfaction'] / 10) +
    0.1 * (df['contract_type'] == 'Month-to-month').astype(int) +
    np.random.normal(0, 0.1, n_samples)
)
df['churn'] = (churn_prob > 0.3).astype(int)

print(f"数据形状: {df.shape}")
print(f"流失率: {df['churn'].mean():.2%}")
print(f"\n数据预览:")
print(df.head())
print(f"\n数据类型:")
print(df.dtypes)
print(f"\n缺失值:")
print(df.isnull().sum())


## 2. 数据探索（EDA）

探索数据的基本特征、分布和相关性。


In [ ]:
# 数据统计信息
print("数值特征统计:")
print(df.describe())

# 类别特征统计
print("\n类别特征统计:")
print(f"合同类型分布:")
print(df['contract_type'].value_counts())
print(f"\n支付方式分布:")
print(df['payment_method'].value_counts())

# 流失率分析
print(f"\n流失率分析:")
print(f"总体流失率: {df['churn'].mean():.2%}")
print(f"\n按合同类型的流失率:")
print(df.groupby('contract_type')['churn'].agg(['mean', 'count']))
print(f"\n按支付方式的流失率:")
print(df.groupby('payment_method')['churn'].agg(['mean', 'count']))


In [ ]:
# 可视化数据分布
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 数值特征分布
numeric_cols = ['age', 'income', 'usage_months', 'complaints', 'service_calls', 'satisfaction']
for i, col in enumerate(numeric_cols):
    row, col_idx = i // 3, i % 3
    axes[row, col_idx].hist(df[col], bins=30, alpha=0.7, edgecolor='black')
    axes[row, col_idx].set_title(f'{col} 分布')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('频数')
    axes[row, col_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# 相关性分析
numeric_df = df.select_dtypes(include=[np.number])
correlation = numeric_df.corr()

# 绘制相关性热力图
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('特征相关性热力图')
plt.tight_layout()
plt.show()

# 流失与各特征的关系
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, col in enumerate(numeric_cols):
    row, col_idx = i // 3, i % 3
    df.boxplot(column=col, by='churn', ax=axes[row, col_idx])
    axes[row, col_idx].set_title(f'{col} vs 流失')
    axes[row, col_idx].set_xlabel('流失')
    axes[row, col_idx].set_ylabel(col)

plt.tight_layout()
plt.show()


## 3. 数据预处理

处理类别特征、标准化数值特征。


In [ ]:
# 准备特征和目标
X = df.drop('churn', axis=1)
y = df['churn']

# 处理类别特征：One-Hot编码
X_encoded = pd.get_dummies(X, columns=['contract_type', 'payment_method'], drop_first=True)

print(f"编码后特征数量: {X_encoded.shape[1]}")
print(f"特征名称: {list(X_encoded.columns)}")

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# 标准化数值特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"训练集流失率: {y_train.mean():.2%}")
print(f"测试集流失率: {y_test.mean():.2%}")


## 4. 模型训练

训练逻辑回归模型，使用交叉验证评估。


In [ ]:
# 训练多个模型进行对比
models = {
    '逻辑回归 (C=1.0)': LogisticRegression(C=1.0, random_state=42, max_iter=1000, solver='lbfgs'),
    '逻辑回归 (C=0.1)': LogisticRegression(C=0.1, random_state=42, max_iter=1000, solver='lbfgs'),
    '逻辑回归 (C=10.0)': LogisticRegression(C=10.0, random_state=42, max_iter=1000, solver='lbfgs'),
}

# 交叉验证评估
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

print("交叉验证结果（5折）:")
print("=" * 60)
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='roc_auc')
    results[name] = {
        'model': model,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    print(f"{name:<25}: AUC = {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

# 选择最佳模型
best_model_name = max(results, key=lambda x: results[x]['cv_mean'])
best_model = results[best_model_name]['model']
print(f"\n最佳模型: {best_model_name}")

# 训练最佳模型
best_model.fit(X_train_scaled, y_train)


## 5. 模型评估

全面评估模型在测试集上的性能。


In [ ]:
# 预测
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# 评估指标
accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_proba)

print(f"测试集性能:")
print(f"准确率: {accuracy:.4f}")
print(f"ROC AUC: {auc_score:.4f}")

# 分类报告
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=['未流失', '流失']))

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print("\n混淆矩阵:")
print(cm)

# ROC曲线和PR曲线
fpr, tpr, _ = roc_curve(y_test, y_proba)
precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)


## 6. 结果可视化

可视化模型评估结果。


In [ ]:
# 可视化结果
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. 混淆矩阵
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['未流失', '流失'],
            yticklabels=['未流失', '流失'], ax=axes[0, 0])
axes[0, 0].set_title('混淆矩阵')
axes[0, 0].set_ylabel('真实标签')
axes[0, 0].set_xlabel('预测标签')

# 2. ROC曲线
axes[0, 1].plot(fpr, tpr, label=f'ROC曲线 (AUC = {auc_score:.2f})', linewidth=2)
axes[0, 1].plot([0, 1], [0, 1], 'k--', label='随机分类器')
axes[0, 1].set_xlabel('假正例率 (FPR)')
axes[0, 1].set_ylabel('真正例率 (TPR)')
axes[0, 1].set_title('ROC曲线')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. PR曲线
axes[1, 0].plot(recall, precision, label=f'PR曲线 (AUC = {pr_auc:.2f})', linewidth=2)
axes[1, 0].set_xlabel('召回率 (Recall)')
axes[1, 0].set_ylabel('精确率 (Precision)')
axes[1, 0].set_title('精确率-召回率曲线')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 特征重要性
feature_importance = pd.DataFrame({
    'feature': X_encoded.columns,
    'coefficient': best_model.coef_[0],
    'abs_coefficient': np.abs(best_model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

top_features = feature_importance.head(10)
axes[1, 1].barh(range(len(top_features)), top_features['abs_coefficient'])
axes[1, 1].set_yticks(range(len(top_features)))
axes[1, 1].set_yticklabels(top_features['feature'])
axes[1, 1].set_xlabel('系数绝对值')
axes[1, 1].set_title('Top 10 特征重要性')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

# 显示特征重要性
print("\n特征重要性（Top 10）:")
print(top_features[['feature', 'coefficient', 'abs_coefficient']])
